<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/larissa_santesso/Aula_8_Exerc%C3%ADcio_Larissa_Santesso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Larissa Antonelli Santesso"
print(f'Meu nome é {nome}')

Meu nome é Larissa Antonelli Santesso


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.7 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Tue May 24 11:39:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check which GPU we are using
!nvidia-smi

Tue May 24 21:26:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokensIds_n = []
        self.y = []
        for text in tqdm_notebook(texts):
            tokens_ids = tokenize(text, tokenizer)
            if len(tokens_ids) < context_size + 1:
                continue
            for i in range(len(tokens_ids)-context_size):
                self.tokensIds_n.append(tokens_ids[i:i+context_size])      
                self.y.append(tokens_ids[i+context_size])
                
    def __len__(self):  
        return len(self.tokensIds_n)

    def __getitem__(self, idx):
        
        return torch.tensor(self.tokensIds_n[idx]).long(), torch.tensor(self.y[idx]).long()

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-26 00:36:31--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   171MB/s    in 0.7s    

2022-05-26 00:36:32 (171 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [ ]:
# Load datasets
context_size = 12

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27601545
valid examples: 81770
test examples: 166426


In [ ]:
from collections import OrderedDict

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super(LanguageModel, self).__init__()

        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # Embedding of the words
        self.embeddings_C = nn.Embedding(vocab_size, embedding_dim)

        # Embedding of the words positions
        self.embeddings_P = nn.Embedding(context_size, embedding_dim) 
        self.dropout_emb = nn.Dropout(p=0.1)

        # linear projections
        self.W_q = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_k = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_v = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.W_0 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        # Linear layer
        hidden_size = 2048
        self.feed_forward = nn.Sequential(OrderedDict([
            ('dense1', nn.Linear(embedding_dim, hidden_size)),
            ('relu1',  nn.ReLU()),
            ('dense2', nn.Linear(hidden_size, embedding_dim)),
            ('drop1',  nn.Dropout(p=0.1))]))

        # Softmax
        self.softmax = nn.Softmax(dim=-1)

        # Layer Norms
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

        # Dropout
        self.dropout = nn.Dropout(p=0.1)

        # Output layer
        self.dense = nn.Linear(embedding_dim, vocab_size, bias = False)

    def forward(self, inputs, debug=False):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]

        ##############################  Arguments of the model  ###################################################
        ## B: batch_size
        ## L: context_size
        ## D: embedding_dim
        ## V: vocab_size

        ##################################    Embeddings    ########################################################

        # inputs shape: (B, L)
        # pos shape: (B, L)

        embeds = self.embeddings_C(inputs) # embeds shape: (B, L, D)
        # self.embeddings_P.weight shape: (L, D)

        X = embeds + self.embeddings_P.weight # X shape: (B, L, D)  # Ideia de chamar Embedding pos de maneira mais eficiente inspirada no notebook de Pedro Gengo
        X = self.dropout_emb(X)

        ################################   Self-Attention   ######################################################
        # Q = X[-1] * Wq
        X_1 = X[:,-1,:].unsqueeze(1) # X[:,-1,:] -> shape: (B, D)
        Q = self.W_q(X_1) # Q shape: (B, 1, D)
        # K = X * Wk
        K = self.W_k(X)     # K shape: (B, L, D)
        # V = X * Wv
        V = self.W_v(X)     # V shape: (B, L, D)
        
        # s = Q*K^T
        # normalized scores: s = s/sqrt(D)
        scores = torch.matmul(Q,torch.transpose(K,1,2))/math.sqrt(self.embedding_dim) # scores shape: (B, 1, L)
        probs = self.softmax(scores) # probs shape: (B, 1, L)
        
        E = torch.matmul(probs, V) # E shape: (B, 1, D)

        # E = E*W0
        E = self.W_0(E)
        E = self.dropout(E)
        E = self.layer_norm1(E+X_1) # added layer norm: to test


        ################################### Feed-Forward  #######################################################
        E_flatten = E.view(batch_size,-1)
        feedfoward = self.feed_forward(E_flatten) # out shape: (B, D)
        out = self.layer_norm2(feedfoward+E_flatten) 
        
        ##################################    Linear     ########################################################
        logits = self.dense(out) # logits shape: (B, V)


        if debug:
            print(f"embeds shape: {embeds.shape}")
            #print(embeds)
            print(f"embeds_pos shape: {self.embeddings_P.weight.shape}")
            #print(self.embeddings_P.weight)
            print(f"X shape: {X.shape}")
            #print(X)
            print(f"X_1 shape: {X_1.shape}")
            print(f"Q shape: {Q.shape}")
            print(f"K shape: {K.shape}")
            print(f"V shape: {V.shape}")
            print(f"scores shape: {scores.shape}")
            print(f"probs shape: {probs.shape}")
            print(f"E shape: {E.shape}")
            print(f"E_flatten shape: {E_flatten.shape}")
            print(f"feedfoward shape: {feedfoward.shape}")
            print(f"logits shape: {logits.shape}")

        return logits

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset, batch_size=2)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu, debug=True).shape

embeds shape: torch.Size([2, 12, 256])
embeds_pos shape: torch.Size([12, 256])
X shape: torch.Size([2, 12, 256])
X_1 shape: torch.Size([2, 1, 256])
Q shape: torch.Size([2, 1, 256])
K shape: torch.Size([2, 12, 256])
V shape: torch.Size([2, 12, 256])
scores shape: torch.Size([2, 1, 12])
probs shape: torch.Size([2, 1, 12])
E shape: torch.Size([2, 1, 256])
E_flatten shape: torch.Size([2, 256])
feedfoward shape: torch.Size([2, 256])
logits shape: torch.Size([2, 29794])


torch.Size([2, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 16571648


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              36254
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 500_000_000
eval_every_steps = 10000
lr = 3e-4
compare=float('inf')

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"model07.pt")
                    with open("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"valid_ppl_model07.txt", 'w') as f:
                        f.write("%s\n" % valid_ppl)
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 34315.87, valid ppl: 29926.51
10000 steps; 10240000 examples so far; train ppl: 389.21, valid ppl: 229.44
20000 steps; 20480000 examples so far; train ppl: 227.70, valid ppl: 189.14
30000 steps; 30720000 examples so far; train ppl: 194.45, valid ppl: 170.74
40000 steps; 40960000 examples so far; train ppl: 175.87, valid ppl: 158.62
50000 steps; 51200000 examples so far; train ppl: 166.97, valid ppl: 149.73
60000 steps; 61440000 examples so far; train ppl: 155.55, valid ppl: 143.35
70000 steps; 71680000 examples so far; train ppl: 149.28, valid ppl: 138.68
80000 steps; 81920000 examples so far; train ppl: 146.37, valid ppl: 133.93
90000 steps; 92160000 examples so far; train ppl: 138.02, valid ppl: 131.39
100000 steps; 102400000 examples so far; train ppl: 136.29, valid ppl: 127.10
110000 steps; 112640000 examples so far; train ppl: 133.09, valid ppl: 124.62
120000 steps; 122880000 examples so far; train ppl: 128.37, valid ppl: 123.19
130000 steps;

In [ ]:
with open("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"valid_ppl_model07.txt") as f:
    txt = list(f)
    compare = float(txt[-1])
    f.close()

In [ ]:
max_examples = 300_000_000
eval_every_steps = 10000
lr = 3e-5
print(compare)

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

model = torch.load("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"model07.pt")
model.to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))
                
                if valid_ppl<compare:
                    compare=valid_ppl
                    torch.save(model, "gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"model07_run02.pt")
                    with open("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"valid_ppl_model07_run02.txt", 'w') as f:
                        f.write("%s\n" % valid_ppl)
                    f.close()

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

103.916538251577
0 steps; 0 examples so far; train ppl: 118.97, valid ppl: 103.91
10000 steps; 10240000 examples so far; train ppl: 98.85, valid ppl: 100.74
20000 steps; 20480000 examples so far; train ppl: 97.79, valid ppl: 100.00
30000 steps; 30720000 examples so far; train ppl: 96.91, valid ppl: 99.63
40000 steps; 40960000 examples so far; train ppl: 96.15, valid ppl: 99.34
50000 steps; 51200000 examples so far; train ppl: 96.33, valid ppl: 99.07
60000 steps; 61440000 examples so far; train ppl: 95.66, valid ppl: 99.00
70000 steps; 71680000 examples so far; train ppl: 95.13, valid ppl: 98.84
80000 steps; 81920000 examples so far; train ppl: 95.78, valid ppl: 98.57
90000 steps; 92160000 examples so far; train ppl: 94.89, valid ppl: 98.71
100000 steps; 102400000 examples so far; train ppl: 94.92, valid ppl: 98.44
110000 steps; 112640000 examples so far; train ppl: 94.85, valid ppl: 98.36
120000 steps; 122880000 examples so far; train ppl: 94.30, valid ppl: 98.36
130000 steps; 13312000

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
# Load datasets
context_size = 12

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

test_texts = texts[-test_examples:]

test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

model = torch.load("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"model07.pt", map_location=device)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 97.39266351132329


In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

model = torch.load("gdrive/MyDrive/Colab Notebooks/modelos_Aula08/"+"model07_run02.pt", map_location=device)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 91.2738856731302


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto muito de comer pizza e chocolate, pois'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto muito de comer pizza e chocolate, pois eu
Eu gosto muito de comer pizza e chocolate, pois eu não
Eu gosto muito de comer pizza e chocolate, pois eu não sei
Eu gosto muito de comer pizza e chocolate, pois eu não sei se
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que eu
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que eu quero
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que eu quero dizer
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que eu quero dizer.
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que o que eu quero dizer..
Eu gosto muito de comer pizza e chocolate, pois eu não sei se é que

In [ ]:
prompt = "A exposição da Tarsila ocorrerá no dia 01 de Maio na"
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo.
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento foi
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento foi realizado
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento foi realizado no
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento foi realizado no dia
A exposição da Tarsila ocorrerá no dia 01 de Maio na cidade de São Paulo. O evento foi realizado no dia 15
A exposiç

In [ ]:
prompt = "Temos que sempre pensar no futuro e guardar o que aprendemos na "
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Temos que sempre pensar no futuro e guardar o que aprendemos na vida
Temos que sempre pensar no futuro e guardar o que aprendemos na vida.
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o homem
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o homem é
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o homem é o
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o homem é o homem
Temos que sempre pensar no futuro e guardar o que aprendemos na vida. O que é que o homem é o